# ETL (Extracción, Transformación y Carga) - Dataset de peliculas

## 1. Preparación inicial de datos

### Imports

In [1]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv('data/movies_dataset.csv') 

  df = pd.read_csv('movies_dataset.csv')


### Eliminar duplicados

In [2]:
df = df.drop_duplicates() # Elimina duplicados por fila

### Verificar valores nulos en el conjunto de datos.


In [3]:
print(df.isnull().sum())

adult                        0
belongs_to_collection    40959
budget                       0
genres                       0
homepage                 37673
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64


En columnas como "belongs_to_collection", los valores nulos pueden ser válidos y no se consideran como valores faltantes, no es necesario eliminar esas filas.

### Reemplazar y eliminar valores nulos de las colmunas relevantes  

In [5]:
df.loc[:, 'revenue'] = df['revenue'].replace(np.nan, 0)
df.loc[:, 'budget'] = df['budget'].replace(np.nan, 0)
df.dropna(subset=['title'], inplace=True)
df.dropna(subset=['release_date'], inplace=True)

## 2. Transformación y manipulación de datos

### Eliminar columnas que no se utilizarán

In [6]:
df = df.drop(['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], axis=1)

### Eliminar filas con valores faltantes en el campo 'release_date' y Convertir 'release_date' al formato de fecha (AAAA-mm-dd)

In [7]:
df.dropna(subset=['release_date'], inplace=True)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

### Extraer el año de la fecha de estreno y crear una nueva columna 'release_year'

In [8]:
df['release_year'] = df['release_date'].dt.year
df['release_year'] = df['release_year'].replace(np.nan, 0)
df['release_year'] = df['release_year'].astype(int)

### Conversión de tipos de datos

In [9]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

### Crear una nueva columna 'return' que calcule el retorno de inversión (revenue / budget)

In [10]:
def calc_return(row):
    if pd.isna(row['revenue']) or pd.isna(row['budget']) or row['budget'] == 0:
        return 0
    return row['revenue'] / row['budget']

df['return'] = df.apply(calc_return, axis=1)
df = df.dropna(subset=['return'])

## 3. Procesamiento de datos anidados

### Procesar datos anidados, y extraer información relevante.

In [11]:
def fetch_name(obj):
    if isinstance(obj, str) and '{' in obj: # Verifica que sea una cadena que contenga una lista de diccionarios
        L = []
        for i in ast.literal_eval(obj): # Convierte el string diccionario, despues itera los diccionario anidados
            L.append(i['name'])
        return L

def fetch_name2(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

df['genres'] = df['genres'].apply(fetch_name)
df = df.dropna(subset=['genres'])

df['production_companies']  = df['production_companies'].apply(fetch_name)
df['production_companies']  = df['production_companies'].fillna('')

df['production_countries']  = df['production_countries'].apply(fetch_name)
df['production_countries']  = df['production_countries'].fillna('')

df['belongs_to_collection'] = df['belongs_to_collection'].apply(fetch_name2) 

## 4. Exportación


### Guardar el conjunto de datos limpio en un nuevo archivo CSV

In [12]:

df.to_csv('data/movies_dataset_cleaned.csv', index=False)

In [13]:
print(df.isnull().sum())

belongs_to_collection        0
budget                       0
genres                       0
id                           0
original_language           10
overview                   694
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    176
spoken_languages             0
status                      50
tagline                  22845
title                        0
vote_average                 0
release_year                 0
return                       0
dtype: int64
